# Capstone Project - The Battle of Neighborhoods (Week 1)

## Where to Go When You Want to Eat Sushi ?

#### Background

Let's say you have never been to Spain and you want to have only sushi while you are there. So you want to go to a place with a high density of Sushi restaurants around you. The problem we aim to solve is to analyze the Sushi restaurants' locations in the major Spain cities and find the best place for our tourist so that he can have a good 'Shushirism'.

#### Data Section

I will use the FourSquare API to collect data about locations of Sushi Restaurants in 5 of Spain's major cities which are: New York,NY, San Francisco, CA, Jersey City, NJ, Boston, MA and Chicago,IL. These are one of the mosr populated US cities and I am hopeful that they will contain the best Pizza places in the US.



In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'J1LIAMG51DN4WHN3RCQ3NTMPV4D51PMOSTG3T0Q51FE5YGX2' # your Foursquare ID
CLIENT_SECRET = '45BBB5DQGA4MV2HAGQ2MS1ZOXEBTOP0AGEKGAP5S02T5KMC0' # your Foursquare Secret
VERSION = '20200805' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: J1LIAMG51DN4WHN3RCQ3NTMPV4D51PMOSTG3T0Q51FE5YGX2


Before using data, we will have to explore and understand it.

#### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows: 

In [15]:
LIMIT = 500 # Maximum is 100
cities = ["Madrid", 'Barcelona', 'Valencia', 'Seville', 'Zaragoza']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1d2941735") # SUSHI RESTAURANTS CATEGORY ID
    results[city] = requests.get(url).json()

In [16]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [17]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Sushi Restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in Madrid =  149
Showing Top 100
Total number of pizza places in Barcelona =  147
Showing Top 100
Total number of pizza places in Valencia =  58
Showing Top 100
Total number of pizza places in Seville =  20
Showing Top 100
Total number of pizza places in Zaragoza =  22
Showing Top 100


In [ ]:
maps[cities[0]]

In [ ]:
maps[cities[1]]

In [ ]:
maps[cities[2]]

In [ ]:
maps[cities[3]]

### We can see that Madrid and Barcelona are the most dense cities with Sushi restaurants.
### However, Let's have a concrete measure of this density.
### For this we will use some basic statistics. I will get the mean location of the Sushi restaurants which should be near to most of them if they are really dense or far if not.
### Next I will take the average of the distance of the venues to the mean coordinates

In [20]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Madrid
Mean Distance from Mean coordinates
0.027811023183463356
Barcelona
Mean Distance from Mean coordinates
0.017011020078559994
Valencia
Mean Distance from Mean coordinates
0.015409726342798944
Seville
Mean Distance from Mean coordinates
0.015853917346167753
Zaragoza
Mean Distance from Mean coordinates
0.01352640101275873


In [22]:
maps[cities[0]]

In [23]:
maps[cities[1]]

In [24]:
maps[cities[2]]

In [25]:
maps[cities[3]]

### We now see that Madrid is his best option. And as aplus the fourth best place is Valencia which is only 2 hours away on train. Our tourist's best interest would be to book a hotel on Madrid and take a 2 day trip to Seville to enjoy as much sushi as he/she can and enjoy Seville's beaches as well !!
Another observation is that there is one really far away Sushi restaurants which would possibly increase Madrid's score.

In [42]:
city = 'Madrid'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

Madrid
Mean Distance from Mean coordinates
0.026946519853024153


In [ ]:
# Eliminating this Restaurant doesnt manage to alter Madrid's score that much, in fact it lowers 